# Two Layered RS Demo

Код для src, utils, metrics вы можете скачать из [этого](https://github.com/geangohn/recsys-tutorial) github репозитория

In [4]:
#############################
### TODO REMOVE CONSTRAINTS:
#############################

"""
- Для каждого юзера 5 рекомендаций (иногда модели могут возвращать < 5)
- 2 новых товара (юзер никогда не покупал)
- 1 дорогой товар, > 7 долларов
- Все товары из разных категорий (категория - sub_commodity_desc)

- Стоимость каждого рекомендованного товара > 1 доллара
"""
None

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
from implicit.als import AlternatingLeastSquares
from implicit.bpr import BayesianPersonalizedRanking
import re
import time
from lightgbm import LGBMClassifier
import os, sys

from src.metrics import precision_at_k
from implicit.nearest_neighbours import ItemItemRecommender, bm25_weight 
# from src.utils import prefilter_items
# from src.recommenders import MainRecommender


In [6]:
###############################
# Settings && Constants 
###############################

DATA_PATH = './data/retail_train.csv'
TEST_PATH = './data/retail_test1.csv'
ITEM_FEATURES_PATH = './data/product.csv'
USER_FEATURES_PATH = './data/hh_demographic.csv'

TEST_SIZE_WEEKS = (3,3)
N_POPULAR_ITEMS = 4000
INIT_NUM_RECS = 500
N_FIN_RECS = 5
# NUM_THREADS = 8

In [47]:
###############################
# MODULES 
###############################

value_string_template = '\033[91m[[value]]\033[0m'

def prefilter_items(data, take_n_popular=5000, margin_slice_rate=0.9):
    
    """Предфильтрация товаров"""
    
    n_before = value_string_template.replace('[[value]]', str(data['item_id'].nunique()))
    
    
    # drop 0 purchases
    data = data.drop(data[data['quantity']==0].index)
    
    # Calculate price
    data['price'] = data['sales_value'] / data['quantity']
    
    # 1. Drop prices < 1$
    data = data[data['price'] > 1]
    
    # Удаление товаров со средней ценой > 15$
    data = data[data['price'] < 15]
    
    # Уберем товары, которые не продавались за последние 5 месяцев
    purchases_last_week = data.groupby('item_id')['week_no'].max().reset_index()
    weeks = purchases_last_week[
        purchases_last_week['week_no'] > data['week_no'].max() - 5].item_id.tolist()
    data = data[data['item_id'].isin(weeks)]
    
    # Удаление 10% товаров c наименьшей выручкой (сдвигает минимум выручки с 1.1$ до 94.8$ для unsplitted data)
    marginality = data.groupby('item_id')['sales_value'].sum().reset_index()
    ten_percent_slice_idx = int(marginality.shape[0] * margin_slice_rate)

    top_margin = marginality.sort_values('sales_value', ascending=False)[:ten_percent_slice_idx].item_id.tolist()
    data = data[data['item_id'].isin(top_margin)]
    
    # Выбор топ-N самых популярных товаров (N = take_n_popular)
    popularity = data.groupby('item_id')['quantity'].sum().reset_index()
    top_popular = popularity.sort_values('quantity', ascending=False)[:take_n_popular].item_id.tolist()
    data = data[data['item_id'].isin(top_popular)]
    
    n_after = value_string_template.replace('[[value]]', str(data['item_id'].nunique()))
    print(f"Items variety reduced from: {n_before} to: {n_after} samples...", end='')
    print('\033[94mDone\033[0m')
    
    return data


def get_raw_data_splits(data_path, n_weeks_split=(3, 3), mode=0):
    
    """
    Return data splits depending on mode:
    
    MODE 0: No split
    MODE 1: One level split
    MODE 2: Two level split
    
    data_train: base train split
    data_test: used for lvl 1 validation & lvl 2 train
    data_val: used for lvl 2 validation
    
    for lvl_size_weeks in [6, 3] returns:   
    train_lvl1: week_no (1-85), val_lvl1 & train_lvl2: week_no (86-91), val_lvl2: week_no (92-95)
    """
    print("Preparing raw data...", end='')
    data = pd.read_csv(data_path)
    
    if mode == 0:
        return data
    
    if mode == 1:
        print("Selected one level mode...", end='')
        data_train = data[data['week_no'] < data['week_no'].max() - (n_weeks_split[0] + n_weeks_split[1])]
        data_test = data[(data['week_no'] >= data['week_no'].max() - (n_weeks_split[0] + n_weeks_split[1]))]
        print('\033[94mDone\033[0m') 
        
        return data, data_train, data_test
    
    elif mode == 2:
        print("Selected two level mode...", end='')
        data_train = data[data['week_no'] < data['week_no'].max() - (n_weeks_split[0] + n_weeks_split[1])]
        data_test = data[(data['week_no'] >= data['week_no'].max() /
                               - (n_weeks_split[0] + n_weeks_split[1])) &
                              (data['week_no'] < data['week_no'].max() - (n_weeks_split[1]))]
        data_val_1 = data_test.copy()
        data_val_2 = data[data['week_no'] >= data['week_no'].max() - n_weeks_split[1]] 
        print('\033[94mDone\033[0m') 

        return data, data_train, data_test, data_val_1, data_val_2
    else:
        print('\033[91mError. Mode not understood\033[0m')
        return None

    
def get_price_list(data_1, data_2, _id='item_id', target='price'):
    
    """
    Used 4 creating check dict (price list)
    From all historical data
    Can be used once and stored then updated if needs
    """
    
    pl_1 = data_1.groupby(_id)[target].mean().reset_index()
    pl_2 = data_2.groupby(_id)[target].mean().reset_index()
    d1 = dict(zip(pl_1[_id], pl_1[target]))
    d2 = dict(zip(pl_2[_id], pl_2[target]))
    pl_emb = {**d1, **d2}
    
    return pl_emb


def get_bought_ever_list(data_1, data_2, _id='user_id', target='item_id'):
    
    """
    Used 4 creating check dict (user's ever bought list)
    From all historical data
    Can be used once and stored then updated if needs
    """
    
    pl_1 = data_1.groupby(_id)[target].unique().reset_index()
    pl_2 = data_2.groupby(_id)[target].unique().reset_index()
    d1 = dict(zip(pl_1[_id], pl_1[target]))
    d2 = dict(zip(pl_2[_id], pl_2[target]))
    pl_emb = {**d1, **d2}
    
    return pl_emb


def get_item_commodities_list(feats, _id='item_id', target='sub_commodity_desc_code'):
    
    """
    Used 4 creating check dict (item_id - commodity_type)
    From all historical data
    Can be used once and stored then updated if needs
    """
    
    res = dict(zip(feats[_id], feats[target]))
    return res
        
    
def preprare_features(item_features_path, user_features_path):
    
    """Loads raw item and user features:"""
    
    print("Preparing raw features...", end='')
    item_features = pd.read_csv(item_features_path)
    user_features = pd.read_csv(user_features_path)

    # column processing
    item_features.columns = [col.lower() for col in item_features.columns]
    user_features.columns = [col.lower() for col in user_features.columns]
    item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
    user_features.rename(columns={'household_key': 'user_id'}, inplace=True)
    
    # encode commodities in item_features
    item_features['sub_commodity_desc'] = pd.Categorical(item_features['sub_commodity_desc'])
    item_features['sub_commodity_desc_code'] = item_features['sub_commodity_desc'].cat.codes
    
    print('\033[94mDone\033[0m')
    
    return item_features, user_features



class MainRecommender:
    """Рекоммендации, которые можно получить из ALS
    Input
    -----
    user_item_matrix: pd.DataFrame
        Матрица взаимодействий user-item
    """

    def __init__(self, data, prices, weighting=True):

        # Топ покупок каждого юзера
        self.top_purchases = data.groupby(['user_id', 'item_id'])['sales_value'].count().reset_index()
        self.top_purchases.sort_values('sales_value', ascending=False, inplace=True)

        # Топ покупок по всему датасету
        self.overall_top_purchases = data.groupby('item_id')['sales_value'].count().reset_index()
        self.overall_top_purchases.sort_values('sales_value', ascending=False, inplace=True)
        self.overall_top_purchases = self.overall_top_purchases.item_id.tolist()

        self.user_item_matrix, self.matrix_index, self.matrix_columns = self._prepare_matrix(data)  # pd.DataFrame
        self.id_to_itemid, self.id_to_userid, \
            self.itemid_to_id, self.userid_to_id = self._prepare_dicts(self.user_item_matrix)

        if weighting:
            self.user_item_matrix = bm25_weight(self.user_item_matrix.T).T

        self.model = self.fit(self.user_item_matrix)
        # self.bpr = self.fit_bpr(self.user_item_matrix)
        self.own_recommender = self.fit_own_recommender(self.user_item_matrix)
        
        self.item_factors = self.model.item_factors
        self.user_factors = self.model.user_factors
#         self.price_list = prices
        
        self.items_emb_df, self.users_emb_df = self.get_embeddings(self)
        
        
    @staticmethod
    def get_embeddings(self):
        items_emb = self.item_factors
        items_emb_df = pd.DataFrame(items_emb)
        items_emb_df.reset_index(inplace=True)
        items_emb_df['item_id'] = items_emb_df['index'].apply(lambda x: self.id_to_itemid[x])
        items_emb_df = items_emb_df.drop('index', axis=1)

        users_emb = self.user_factors
        users_emb_df = pd.DataFrame(users_emb)
        users_emb_df.reset_index(inplace=True)
        users_emb_df['user_id'] = users_emb_df['index'].apply(lambda x: self.id_to_userid[x])
        users_emb_df = users_emb_df.drop('index', axis=1)

        return items_emb_df, users_emb_df
        

    @staticmethod
    def _prepare_matrix(data):
        """Готовит user-item матрицу"""
        user_item_matrix = pd.pivot_table(data,
                                          index='user_id', 
                                          columns='item_id',
                                          values='quantity',
                                          aggfunc='count',
                                          fill_value=0
                                          )
        matrix_index = user_item_matrix.index
        matrix_columns = user_item_matrix.columns

        user_item_matrix = user_item_matrix.astype(float)  # необходимый тип матрицы для implicit
        return user_item_matrix, matrix_index, matrix_columns
    

    @staticmethod
    def _prepare_dicts(user_item_matrix):
        """Подготавливает вспомогательные словари"""

        userids = user_item_matrix.index.values
        itemids = user_item_matrix.columns.values

        matrix_userids = np.arange(len(userids))
        matrix_itemids = np.arange(len(itemids))

        id_to_itemid = dict(zip(matrix_itemids, itemids))
        id_to_userid = dict(zip(matrix_userids, userids))

        itemid_to_id = dict(zip(itemids, matrix_itemids))
        userid_to_id = dict(zip(userids, matrix_userids))

        return id_to_itemid, id_to_userid, itemid_to_id, userid_to_id
    

    @staticmethod
    def fit_own_recommender(user_item_matrix):
        
        """Обучает модель, которая рекомендует товары, среди товаров, купленных юзером"""
        
        own_recommender = ItemItemRecommender(K=1)
        own_recommender.fit(csr_matrix(user_item_matrix).T.tocsr())
        return own_recommender
    

    @staticmethod
    def fit(user_item_matrix, n_factors=32, regularization=0.001, iterations=20, num_threads=8):
        
        """Обучает ALS"""
        
        model = AlternatingLeastSquares(factors=n_factors,
                                        regularization=regularization,
                                        iterations=iterations,
                                        num_threads=num_threads)
        model.fit(csr_matrix(user_item_matrix).T.tocsr())
        return model
    
    @staticmethod
    def fit_bpr(user_item_matrix, n_factors=31, regularization=0.001, iterations=20, num_threads=8):
        model = BayesianPersonalizedRanking(factors=n_factors,
                                             regularization=regularization,
                                             iterations=iterations,
                                             num_threads=num_threads)
        model.fit(csr_matrix(user_item_matrix).T.tocsr())
        return model

    def _update_dict(self, user_id):
        
        """Если появился новый user / item, то нужно обновить словари"""
        
        if user_id not in self.userid_to_id.keys():
            print(f"user_id: '\033[94m{user_id}\033[0m' not in dict, add...")
            max_id = max(list(self.userid_to_id.values()))
            max_id += 1
            self.userid_to_id.update({user_id: max_id})
            self.id_to_userid.update({max_id: user_id})
            

    def _get_similar_item(self, item_id):
        
        """Находит товар, похожий на item_id"""
        
        # Товар похож на себя -> рекомендуем 2 товара
        recs = self.model.similar_items(self.itemid_to_id[item_id], N=2)
        top_rec = recs[1][0]  # И берем второй (не товар из аргумента метода)
        return self.id_to_itemid[top_rec]
    

    def _extend_with_top_popular(self, recommendations, N=5):
        
        """Если кол-во рекоммендаций < N, то дополняем их топ-популярными"""
        
        if len(recommendations) < N:
            recommendations.extend(self.overall_top_purchases[:N])
            recommendations = recommendations[:N]
        return recommendations
    

    def _get_recommendations(self, user, model, N=5):
        
        """Рекомендации через стардартные библиотеки implicit"""
        
        
        if user not in self.userid_to_id.keys():
            self._update_dict(user_id=user)
            res = []
            res = self._extend_with_top_popular(res, N=N)
        else:
        
        
            res = [self.id_to_itemid[rec[0]] for rec in model.recommend(userid=self.userid_to_id[user],
                                            user_items=csr_matrix(self.user_item_matrix).tocsr(),
                                            N=N,
                                            filter_already_liked_items=False,
                                            filter_items=None,
                                            recalculate_user=True)]
            res = self._extend_with_top_popular(res, N=N)
        assert len(res) == N, 'Количество рекомендаций != {}'.format(N)
        return res
    

    def get_als_recommendations(self, user, N=5):
        
        """Рекомендации через стардартные библиотеки implicit"""
        
#         self._update_dict(user_id=user)
        return self._get_recommendations(user, model=self.model, N=N)
    
    def get_bayesian_recommendations(self, user, N=5):
        return self._get_recommendations(user, model=self.bpr, N=N)

    def get_own_recommendations(self, user, N=5):
        
        """Рекомендуем товары среди тех, которые юзер уже купил"""
        
#         self._update_dict(user_id=user)
        return self._get_recommendations(user, model=self.own_recommender, N=N)
    

    def get_similar_items_recommendation(self, user, N=5):
        
        """Рекомендуем товары, похожие на топ-N купленных юзером товаров"""
        
        top_users_purchases = self.top_purchases[self.top_purchases['user_id'] == user].head(N)

        res = top_users_purchases['item_id'].apply(lambda x: self._get_similar_item(x)).tolist()
        res = self._extend_with_top_popular(res, N=N)
        assert len(res) == N, 'Количество рекомендаций != {}'.format(N)
        return res
    

    def get_similar_users_recommendation(self, user, N=5):
        
        """Рекомендуем топ-N товаров, среди купленных похожими юзерами"""
        
        res = []

        # Находим топ-N похожих пользователей
        similar_users = self.model.similar_users(self.userid_to_id[user], N=N+1)
        similar_users = [rec[0] for rec in similar_users]
        similar_users = similar_users[1:]   # удалим юзера из запроса

        for user in similar_users:
            res.extend(self.get_own_recommendations(user, N=1))


        res = self._extend_with_top_popular(res, N=N)
        assert len(res) == N, 'Количество рекомендаций != {}'.format(N)
        return res
    
    
def check_valid_items(res,
                      user,
                      item_to_commodity,
                      itemid_to_price,
                      user_bought_history,
                      n=5,
                      max_price_constraint=7,
                      max_n_new_items_constraint=2
                     ):
    
    """
    Constraints Checker Module.
    Check commended items for user.
    Input: res:array with len=n
    Return: error flag, one-hot encodded errors
    """
    
    # res: list result of n=5 elements
    
    err_flag = 0  # error flag
    err_pos = np.zeros(len(res))  # penalty weights for items to change positions
    user_history = user_bought_history[user][:10]  # user purchases of unique items history
    price_checklist = np.zeros(len(res))  # price list for checking conditions
    unique_checklist = np.zeros(len(res))  # unique item positions counter
    
    c = []
    for i, item in enumerate(res):
        commodity = item_to_commodity[item]
        price_checklist[i] = itemid_to_price[item]        
        
        # check if code already exists in recs. If true change first duplicate
        if commodity in c:
            err_flag = 1
            err_pos[i] += 1  # add penalty for mismatch element position
            
        # keep track of unique items
        if item not in user_history:
            unique_checklist[i] = 1
        c.append(commodity)
        
    max_price = price_checklist[np.argmax(price_checklist)]
    
    # if no expensive items in list, mark last element
    if max_price < max_price_constraint:
        err_flag = 1
        
        # add penalty to last highest element
        #  len(res-1) - np.argmax(err_pos[::-1]) keeps track on the last element if equal weights exist like 0
        #  used in case of ranked elements
        err_pos[len(res)-1 - np.argmax(err_pos[::-1])] += 1
    
    # if not enough unique elements, mark last element
    if unique_checklist.sum() < max_n_new_items_constraint:
        err_flag = 1
        # add penalty to last highest element
        err_pos[len(res)-1 - np.argmax(err_pos[::-1])] += 1
    
    return err_flag, err_pos

            

def postfilter_items(data,
                     pop_recs,
                     item_to_commodity,
                     itemid_to_price,
                     user_bought_history,
                     n=5,
                     target_col='recomendations',
                     res_col='result'
                    ):    
    
    """
    Input: user recommendations: pd.DataFrame
    """
    # make result placeholders
    data[res_col] = np.nan
    data[res_col] = data[res_col].astype('object')
    
    
    for index, row in data.iterrows():       
        if row.user_id %500==0:
            print(f"iter on {row.user_id}")
        
        recs = row[target_col]
        result = recs[:n] # list with n item_id recs
        flag, err_positions = check_valid_items(result,
                                                row.user_id,
                                                item_to_commodity,
                                                itemid_to_price,
                                                user_bought_history,
                                                n)

        take_from_pos = n + 1  # set initial position of new element to substitute as next # after existing recs
        ov=1
        while flag:
            
            pos_list = np.where(err_positions>0)[0]  # invalid element positions pointer
            # change each invalid element 
            for pos in pos_list:
                if take_from_pos%INIT_NUM_RECS==0 and not ov:  # INIT_NUM_RECS
#                     print(take_from_pos)
                    take_from_pos = 1
                    ov=1
                    
                if ov:
                    result[pos] = pop_recs[take_from_pos]
                    take_from_pos +=1
                    
                else:
                    result[pos] = recs[take_from_pos]
                    take_from_pos +=1
                
            # check new recommendations
            flag, err_positions = check_valid_items(result,
                                                row.user_id,
                                                item_to_commodity,
                                                itemid_to_price,
                                                user_bought_history,
                                                n)        
                    
        data.at[index, res_col] = result
    return data



def money_precision_at_k(recommended_list, bought_list, prices, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list[:k])
    
    prices_recommended = np.array([prices[i] for i in recommended_list])
    prices_bought = np.array([prices[i] for i in bought_list])
    
    flags = np.isin(bought_list, recommended_list)
    precision = np.dot(flags, prices_bought) / np.dot(np.ones(k), prices_recommended)
    
    
    return precision

In [18]:
%%time

# weeks split: 1-88, 89-95, 96-98
data, data_train_lvl_1, data_val_lvl_1 = get_raw_data_splits(DATA_PATH, mode=1)

data_train_lvl_2 = data_val_lvl_1.copy()
data_val_lvl_2 = get_raw_data_splits(TEST_PATH, mode=0)
# TWO LVL PREPARATION

# data, data_train_lvl_1, data_val_lvl_1, data_train_lvl_2, data_val_lvl_2 = get_raw_data_splits(
#     DATA_PATH, TEST_SIZE_WEEKS, mode=1)

item_features, user_features = preprare_features(ITEM_FEATURES_PATH, USER_FEATURES_PATH)

Preparing raw data...Selected one level mode...Done
Preparing raw data...Preparing raw features...Done
Wall time: 5.34 s


In [22]:
# Prefilter routine
data_train_lvl_1 = prefilter_items(data_train_lvl_1, N_POPULAR_ITEMS) # Prefilter routine

data_val_lvl_1 = prefilter_items(data_val_lvl_1, N_POPULAR_ITEMS) # Prefilter routine
data_train_lvl_2 = prefilter_items(data_val_lvl_1, N_POPULAR_ITEMS) # Prefilter routine
data_val_lvl_2 = prefilter_items(data_val_lvl_1, N_POPULAR_ITEMS) # Prefilter routine

Items variety reduced from: 85334 to: 4000 samples...Done
Items variety reduced from: 30040 to: 4000 samples...Done
Items variety reduced from: 4000 to: 3600 samples...Done
Items variety reduced from: 4000 to: 3600 samples...Done


In [23]:
%%time

# Get avg historical prices for all products
itemid_to_price = get_price_list(data_train_lvl_1, data_val_lvl_1)
user_bought_history = get_bought_ever_list(data_train_lvl_1, data_val_lvl_1)
item_to_commodity =  get_item_commodities_list(item_features)

# data.groupby('item_id')['sales_value'].sum().reset_index()

Wall time: 1.55 s


In [26]:
%%time

recommender = MainRecommender(data_train_lvl_1, itemid_to_price)


Wall time: 8.45 s


In [40]:
train_users = data_train_lvl_1['user_id'].unique()

In [27]:
users_lvl_2 = pd.DataFrame(data_train_lvl_2['user_id'].unique())
users_lvl_2.columns = ['user_id']
users_lvl_2['candidates'] = users_lvl_2['user_id'].apply(lambda x: recommender.get_own_recommendations(x, N=200))

user_id: '296' not in dict, add...
user_id: '2404' not in dict, add...
user_id: '1987' not in dict, add...
user_id: '2259' not in dict, add...


In [28]:
s = users_lvl_2.apply(lambda x: pd.Series(x['candidates']), axis=1).stack().reset_index(level=1, drop=True)
s.name = 'item_id'

users_lvl_2 = users_lvl_2.drop('candidates', axis=1).join(s)
users_lvl_2['drop'] = 1 

In [30]:
targets_lvl_2 = data_train_lvl_2[['user_id', 'item_id']].copy()
targets_lvl_2['target'] = 1  # тут только покупки 

targets_lvl_2 = users_lvl_2.merge(targets_lvl_2, on=['user_id', 'item_id'], how='left')

targets_lvl_2['target'].fillna(0, inplace= True)
targets_lvl_2.drop('drop', axis=1, inplace=True)

In [31]:
"""Feats Preparation"""


### USER FEATS

combined_user_features = user_features.merge(data_train_lvl_2, on='user_id', how='left')

# Income feature prepare
income_desc_lst = list(user_features.income_desc.unique())
income_desc_2_income = [int(re.search(r'\d{2,3}', l)[0]) for l in income_desc_lst]
income_dict = dict(zip(income_desc_lst, income_desc_2_income))

user_features['income, x1000 $'] = user_features['income_desc'].replace(income_dict)
user_features.drop('income_desc', axis=1, inplace=True)


# Age feature prepare
age_desc_lst = list(user_features.age_desc.unique())
age_desc_2_age = [int(re.search(r'\d{2}', l)[0]) for l in age_desc_lst]
age_dict = dict(zip(age_desc_lst, age_desc_2_age))

user_features['age'] = user_features['age_desc'].replace(age_dict)
user_features.drop('age_desc', axis=1, inplace=True)

# Combined feats

# Mean сheck
user_check = combined_user_features.groupby(['user_id'])['sales_value'].sum().reset_index()
user_check_cnt = combined_user_features.groupby('user_id')['basket_id'].count().reset_index()
user_check_cnt.rename(columns={'basket_id': 'baskets_cnt'}, inplace=True)

average_check = user_check.merge(user_check_cnt)

average_check['avg_check'] = average_check['sales_value'] / average_check['baskets_cnt']
average_check = average_check.drop(['sales_value', 'baskets_cnt'], axis=1)
user_features = user_features.merge(average_check, how='left')


### ITEM FEATS

combined_item_features = item_features.merge(data_train_lvl_2, on='item_id', how='left')


### Sold cound
item_count = combined_item_features.groupby(['item_id'])['quantity'].count().reset_index()
item_count.rename(columns={'quantity': 'sales_quantity'}, inplace=True)

item_count['sales_quantity_p_week'] = item_count['sales_quantity'] / combined_item_features['week_no'].nunique()
item_features = item_features.merge(item_count, on='item_id', how='left')

In [32]:
print(f'Incomes dict: {income_dict}\n')
print(f'Ages dict: {age_dict}')

Incomes dict: {'35-49K': 35, '50-74K': 50, '25-34K': 25, '75-99K': 75, 'Under 15K': 15, '100-124K': 100, '15-24K': 15, '125-149K': 125, '150-174K': 150, '250K+': 250, '175-199K': 175, '200-249K': 200}

Ages dict: {'65+': 65, '45-54': 45, '25-34': 25, '35-44': 35, '19-24': 19, '55-64': 55}


In [33]:
targets_lvl_2 = targets_lvl_2.merge(item_features, on='item_id', how='left')
targets_lvl_2 = targets_lvl_2.merge(user_features, on='user_id', how='left')

In [34]:
targets_lvl_2['target'].mean()

0.08448979968955576

In [35]:
X_train = targets_lvl_2.drop('target', axis=1)
y_train = targets_lvl_2[['target']]

In [36]:
cat_feats = X_train.columns[2:].tolist()
X_train[cat_feats] = X_train[cat_feats].astype('category')

cat_feats

['manufacturer',
 'department',
 'brand',
 'commodity_desc',
 'sub_commodity_desc',
 'curr_size_of_product',
 'sub_commodity_desc_code',
 'sales_quantity',
 'sales_quantity_p_week',
 'marital_status_code',
 'homeowner_desc',
 'hh_comp_desc',
 'household_size_desc',
 'kid_category_desc',
 'income, x1000 $',
 'age',
 'avg_check']

In [37]:
%%time

lgb = LGBMClassifier(objective='binary', max_depth=5, categorical_column=cat_feats)
lgb.fit(X_train, y_train)

C:\Users\Nickel\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Nickel\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Nickel\anaconda3\lib\site-packages\lightgbm\basic.py:814: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Wall time: 5.02 s


LGBMClassifier(boosting_type='gbdt',
               categorical_column=['manufacturer', 'department', 'brand',
                                   'commodity_desc', 'sub_commodity_desc',
                                   'curr_size_of_product',
                                   'sub_commodity_desc_code', 'sales_quantity',
                                   'sales_quantity_p_week',
                                   'marital_status_code', 'homeowner_desc',
                                   'hh_comp_desc', 'household_size_desc',
                                   'kid_category_desc', 'income, x1000 $',
                                   'age', 'avg_check'],
               class_weight=None, colsample_bytree=1.0, importance_type='split',
               learning_rate=0.1, max_depth=5, min_child_samples=20,
               min_child_weight=0.001, min_split_gain=0.0, n_estimators=100,
               n_jobs=-1, num_leaves=31, objective='binary', random_state=None,
               reg_alpha=0.

In [38]:
X_train.columns

Index(['user_id', 'item_id', 'manufacturer', 'department', 'brand',
       'commodity_desc', 'sub_commodity_desc', 'curr_size_of_product',
       'sub_commodity_desc_code', 'sales_quantity', 'sales_quantity_p_week',
       'marital_status_code', 'homeowner_desc', 'hh_comp_desc',
       'household_size_desc', 'kid_category_desc', 'income, x1000 $', 'age',
       'avg_check'],
      dtype='object')

In [41]:
### Validation prep


result_lvl_2 = data_val_lvl_2.groupby('user_id')['item_id'].unique().reset_index()
result_lvl_2.columns=['user_id', 'actual']


val_users_lvl_2 = pd.DataFrame(data_val_lvl_2['user_id'].unique())
val_users_lvl_2.columns = ['user_id']
# Warm start
# train_lvl_2_users = data_train_lvl_1['user_id'].unique()
val_users_lvl_2 = val_users_lvl_2[val_users_lvl_2['user_id'].isin(train_users)]

val_users_lvl_2['candidates'] = val_users_lvl_2['user_id'].apply(
    lambda x: recommender.get_own_recommendations(x, N=200))



s2 = val_users_lvl_2.apply(lambda x: pd.Series(x['candidates']), axis=1).stack().reset_index(level=1, drop=True)
s2.name = 'item_id'
val_users_lvl_2 = val_users_lvl_2.drop('candidates', axis=1).join(s)
val_users_lvl_2['drop'] = 1 


val_targets_lvl_2 = data_val_lvl_2[['user_id', 'item_id']].copy()
val_targets_lvl_2['target'] = 1  # тут только покупки 

val_targets_lvl_2 = val_users_lvl_2.merge(val_targets_lvl_2, on=['user_id', 'item_id'], how='left')

val_targets_lvl_2.dropna(0, subset=['item_id'],inplace=True)
val_targets_lvl_2['item_id'] = val_targets_lvl_2['item_id'].astype('int64')


val_targets_lvl_2['target'].fillna(0, inplace= True)
val_targets_lvl_2.drop('drop', axis=1, inplace=True)


val_targets_lvl_2 = val_targets_lvl_2.merge(item_features, on='item_id', how='left')
val_targets_lvl_2 = val_targets_lvl_2.merge(user_features, on='user_id', how='left')

print(val_targets_lvl_2['target'].mean())

X_val = val_targets_lvl_2.drop('target', axis=1)

y_val = targets_lvl_2[['target']]

0.0846277029028436


In [42]:
X_val[cat_feats] = X_val[cat_feats].astype('category')

val_preds = lgb.predict_proba(X_val)[:, 1]
X_val['predict'] = val_preds
X_val.sort_values(['user_id', 'predict'], ascending=False, inplace=True)
result = X_val.groupby('user_id').head(200)

In [43]:
recs = result.groupby('user_id')['item_id']
recomendations = []
for user, preds in recs:
    recomendations.append({'user_id': user, 'recomendations': preds.tolist()})
    
recomendations = pd.DataFrame(recomendations)

In [44]:
# hot start correction

result_lvl_2 = result_lvl_2.merge(recomendations, how='left')
result_lvl_2.dropna(0, subset=['recomendations'], inplace=True)

In [45]:
result_lvl_2

,user_id,actual,recomendations
0,1,"[829323, 940947, 968164, 995242, 1082185, 1082...","[8293439, 8293439, 8293439, 8293439, 1106523, ..."
1,2,"[821083, 828106, 833025, 885023, 899624, 90332...","[957232, 1106523, 1106523, 1029743, 1082185, 1..."
2,3,"[920626, 1053690, 13842214]","[13842214, 13945244, 1013928, 9526628, 1110244..."
3,4,"[831063, 883932, 891423, 908283, 962229, 10213...","[1106523, 1029743, 1029743, 1082185, 1126899, ..."
4,6,"[850102, 897088, 1062782, 1037863, 1119051, 12...","[1098844, 1015296, 1015296, 1119051, 1119051, ..."
...,...,...,...
2105,2496,"[824915, 899624, 900208, 907631, 916122, 92010...","[1065593, 832678, 5569230, 6534035, 965842, 90..."
2106,2497,"[857503, 896613, 1024306, 870515, 897125, 9706...","[835243, 6533936, 854754, 12731436, 1081189, 9..."
2107,2498,"[839243, 951197, 1070820, 1100379, 871337, 918...","[1070820, 1070820, 1085604, 901776, 901776, 11..."
2108,2499,"[853354, 882308, 830887, 833458, 865992, 89962...","[1085604, 1070820, 1070820, 13842090, 13945244..."


In [48]:
%%time
result_lvl_2 = postfilter_items(result_lvl_2,
                            recommender.overall_top_purchases,
                            item_to_commodity,
                            itemid_to_price,
                            user_bought_history,
                            n=N_FIN_RECS)

iter on 1000
iter on 2000
iter on 2500
Wall time: 25.5 s


In [49]:
result_lvl_2

,user_id,actual,recomendations,result
0,1,"[829323, 940947, 968164, 995242, 1082185, 1082...","[8293439, 8293439, 8293439, 8293439, 1106523, ...","[8293439, 981760, 5569230, 1126899, 874972]"
1,2,"[821083, 828106, 833025, 885023, 899624, 90332...","[957232, 1106523, 1106523, 1029743, 1082185, 1...","[957232, 1106523, 981760, 5569230, 874972]"
2,3,"[920626, 1053690, 13842214]","[13842214, 13945244, 1013928, 9526628, 1110244...","[13842214, 13945244, 1013928, 9526628, 1110244]"
3,4,"[831063, 883932, 891423, 908283, 962229, 10213...","[1106523, 1029743, 1029743, 1082185, 1126899, ...","[1106523, 981760, 5569230, 1082185, 874972]"
4,6,"[850102, 897088, 1062782, 1037863, 1119051, 12...","[1098844, 1015296, 1015296, 1119051, 1119051, ...","[1098844, 1015296, 981760, 1119051, 874972]"
...,...,...,...,...
2105,2496,"[824915, 899624, 900208, 907631, 916122, 92010...","[1065593, 832678, 5569230, 6534035, 965842, 90...","[1065593, 832678, 5569230, 981760, 874972]"
2106,2497,"[857503, 896613, 1024306, 870515, 897125, 9706...","[835243, 6533936, 854754, 12731436, 1081189, 9...","[835243, 6533936, 854754, 12731436, 874972]"
2107,2498,"[839243, 951197, 1070820, 1100379, 871337, 918...","[1070820, 1070820, 1085604, 901776, 901776, 11...","[1070820, 981760, 1085604, 901776, 874972]"
2108,2499,"[853354, 882308, 830887, 833458, 865992, 89962...","[1085604, 1070820, 1070820, 13842090, 13945244...","[1085604, 1070820, 981760, 13842090, 874972]"


In [50]:
# Precision compare

# one_lvl_precision = result_lvl_1.apply(lambda row: precision_at_k(row['als_rec'], row['actual'], k=200), axis=1).mean()

res = result_lvl_2.apply(
    lambda row: money_precision_at_k(row['result'], row['actual'], itemid_to_price, k=5), axis=1).mean()

In [51]:
res

0.14146137433787775